<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')


Version:  2.15.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling tensorflow_datasets\horses_or_humans\3.0.0.incompleteGAR1FR\horses_or_humans-train.tfrecord*...:   0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling tensorflow_datasets\horses_or_humans\3.0.0.incompleteGAR1FR\horses_or_humans-test.tfrecord*...:   0%…

Dataset horses_or_humans downloaded and prepared to ./tensorflow_datasets\horses_or_humans\3.0.0. Subsequent calls will reuse this data.


In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [ ]:
!rm -rf logs

In [9]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10



26/26 [==============================] - 5s 146ms/step - loss: 0.6817 - accuracy: 0.5377 - val_loss: 0.6926 - val_accuracy: 0.4341
Epoch 2/10
26/26 [==============================] - 3s 108ms/step - loss: 0.6599 - accuracy: 0.6058 - val_loss: 0.6552 - val_accuracy: 0.6878
Epoch 3/10
26/26 [==============================] - 3s 107ms/step - loss: 0.6313 - accuracy: 0.7275 - val_loss: 0.6571 - val_accuracy: 0.5122
Epoch 4/10
26/26 [==============================] - 3s 110ms/step - loss: 0.5929 - accuracy: 0.7445 - val_loss: 0.5948 - val_accuracy: 0.6537
Epoch 5/10
26/26 [==============================] - 3s 108ms/step - loss: 0.5386 - accuracy: 0.7506 - val_loss: 0.5850 - val_accuracy: 0.6341
Epoch 6/10
26/26 [==============================] - 4s 160ms/step - loss: 0.4894 - accuracy: 0.7701 - val_loss: 0.6109 - val_accuracy: 0.6098
Epoch 7/10
26/26 [==============================] - 3s 135ms/step - loss: 0.4105 - accuracy: 0.8382 - val_loss: 0.3618 - val_accuracy: 0.8732
Epoch 8/10
26/26 

In [ ]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 1: saving model to weights.01-0.67.h5
26/26 - 3s - loss: 0.6738 - accuracy: 0.5876 - val_loss: 0.6694 - val_accuracy: 0.5610 - 3s/epoch - 127ms/step
Epoch 2/5


c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: saving model to weights.02-0.72.h5
26/26 - 3s - loss: 0.6417 - accuracy: 0.6448 - val_loss: 0.7194 - val_accuracy: 0.4732 - 3s/epoch - 104ms/step
Epoch 3/5

Epoch 3: saving model to weights.03-0.60.h5
26/26 - 3s - loss: 0.6238 - accuracy: 0.6606 - val_loss: 0.5970 - val_accuracy: 0.6927 - 3s/epoch - 102ms/step
Epoch 4/5

Epoch 4: saving model to weights.04-0.60.h5
26/26 - 3s - loss: 0.5856 - accuracy: 0.7165 - val_loss: 0.5958 - val_accuracy: 0.6537 - 3s/epoch - 101ms/step
Epoch 5/5

Epoch 5: saving model to weights.05-0.50.h5
26/26 - 3s - loss: 0.5311 - accuracy: 0.7482 - val_loss: 0.5008 - val_accuracy: 0.8000 - 3s/epoch - 100ms/step


In [11]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 1: saving model to saved_model
INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


26/26 - 4s - loss: 0.6871 - accuracy: 0.5377 - val_loss: 0.6667 - val_accuracy: 0.6585 - 4s/epoch - 167ms/step


In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 1: saving model to model.h5
26/26 - 3s - loss: 0.6839 - accuracy: 0.5474 - val_loss: 0.6778 - val_accuracy: 0.5951 - 3s/epoch - 121ms/step
Epoch 2/2


c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: saving model to model.h5
26/26 - 3s - loss: 0.6622 - accuracy: 0.6618 - val_loss: 0.6809 - val_accuracy: 0.4488 - 3s/epoch - 107ms/step


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 3s - loss: 0.6755 - accuracy: 0.5718 - val_loss: 0.6323 - val_accuracy: 0.6341 - 3s/epoch - 116ms/step
Epoch 2/50
26/26 - 2s - loss: 0.5993 - accuracy: 0.6788 - val_loss: 0.5540 - val_accuracy: 0.8537 - 2s/epoch - 88ms/step
Epoch 3/50
26/26 - 2s - loss: 0.5373 - accuracy: 0.7640 - val_loss: 0.6838 - val_accuracy: 0.5854 - 2s/epoch - 94ms/step
Epoch 4/50
26/26 - 2s - loss: 0.4538 - accuracy: 0.8151 - val_loss: 0.4061 - val_accuracy: 0.8341 - 2s/epoch - 93ms/step
Epoch 5/50
26/26 - 2s - loss: 0.3953 - accuracy: 0.8431 - val_loss: 0.4250 - val_accuracy: 0.7756 - 2s/epoch - 91ms/step
Epoch 6/50
26/26 - 2s - loss: 0.3828 - accuracy: 0.8236 - val_loss: 0.3618 - val_accuracy: 0.8488 - 2s/epoch - 92ms/step
Epoch 7/50
26/26 - 3s - loss: 0.2846 - accuracy: 0.9027 - val_loss: 0.2340 - val_accuracy: 0.9463 - 3s/epoch - 123ms/step
Epoch 8/50
26/26 - 3s - loss: 0.2498 - accuracy: 0.9148 - val_loss: 0.1971 - val_accuracy: 0.9463 - 3s/epoch - 131ms/step
Epoch 9/50
26/26 - 4s - loss:

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 3s 95ms/step - loss: 0.6756 - accuracy: 0.6058 - val_loss: 0.7002 - val_accuracy: 0.4390
Epoch 2/5
26/26 [==============================] - 3s 99ms/step - loss: 0.6398 - accuracy: 0.6594 - val_loss: 0.6219 - val_accuracy: 0.7073
Epoch 3/5
26/26 [==============================] - 3s 101ms/step - loss: 0.6036 - accuracy: 0.7263 - val_loss: 0.5749 - val_accuracy: 0.7610
Epoch 4/5
26/26 [==============================] - 3s 100ms/step - loss: 0.5518 - accuracy: 0.7445 - val_loss: 0.5140 - val_accuracy: 0.7610
Epoch 5/5
26/26 [==============================] - 2s 94ms/step - loss: 0.4953 - accuracy: 0.7762 - val_loss: 0.7426 - val_accuracy: 0.5659


In [15]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.605839,0.675560,0.439024,0.700232
1,1,0.659367,0.639813,0.707317,0.621923
2,2,0.726277,0.603587,0.760976,0.574933
3,3,0.744526,0.551809,0.760976,0.514020
4,4,0.776156,0.495294,0.565854,0.742581


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 1: LearningRateScheduler setting learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 3s 99ms/step - loss: 0.6764 - accuracy: 0.5803 - val_loss: 0.6638 - val_accuracy: 0.6585 - lr: 0.0050

Epoch 2: LearningRateScheduler setting learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 2s 89ms/step - loss: 0.6542 - accuracy: 0.6326 - val_loss: 0.6500 - val_accuracy: 0.7561 - lr: 0.0025

Epoch 3: LearningRateScheduler setting learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 3s 119ms/step - loss: 0.6417 - accuracy: 0.6861 - val_loss: 0.6448 - val_accuracy: 0.7366 - lr: 0.0012

Epoch 4: LearningRateScheduler setting learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 4s 141ms/step - loss: 0.6367 - accuracy: 0.7141 - val_loss: 0.6469 - val_accuracy: 0.6585 - lr: 6.2500e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 4

In [ ]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

In [ ]:
%tensorboard --logdir log_dir